<a href="https://colab.research.google.com/github/aminecloud9/PFE/blob/main/CICIDS2017_LSTM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
 
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer,StandardScaler
import h5py
from keras import callbacks
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

Download dataset

In [2]:
data=pd.read_csv('/content/drive/MyDrive/globalData.csv')
#Labels=pd.read_csv('/content/drive/MyDrive/CICIDS2017_Labels.csv')

In [ ]:
print(data.shape)
# get number of unique values for each column
counts = data.nunique()
# record columns to delete
to_del = [i for i,v in enumerate(counts) if v == 1]
print(to_del)
# drop useless columns
for i in enumerate(to_del) :
  data.drop(data.columns[i[1]], axis=1, inplace=True)
print(data.shape)

In [ ]:
del(t)
#del(Labels)

In [ ]:
t=data.values[:,-1]
t.shape

In [8]:
pd.get_dummies(t)

,BENIGN,Bot,DDoS,DoS GoldenEye,DoS Hulk,DoS Slowhttptest,DoS slowloris,FTP-Patator,Heartbleed,Infiltration,PortScan,SSH-Patator,Web Attack � Brute Force,Web Attack � Sql Injection,Web Attack � XSS
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830738,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2830739,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2830740,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2830741,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Split dateset

In [ ]:
#Import Module
from sklearn.model_selection import train_test_split


train_X, test_X, train_y, test_y = train_test_split(data.values[:,:-1], data.values[:,-1], 
                                                     train_size=0.7,
                                                     test_size=0.3,
                                                     random_state=122)

In [ ]:
train_X[:,pd.isna(train_X).any()].tolist()

In [ ]:
scaler = StandardScaler()
train_X= np.nan_to_num(train_X)
X_train = scaler.fit_transform(X=train_X)
X_test = scaler.fit_transform(test_X)

In [ ]:
# reshape input to be [samples, time steps, features]
import keras

#X_train = np.reshape(data.values[:,:-1].astype('float32'), (data.values[:,:-1].shape[0], 1, data.values[:,:-1].shape[1]))
X_train = np.reshape(train_X.astype('float32'), (train_X.shape[0], 1, train_X.shape[1]))
X_test = np.reshape(test_X.astype('float32'), (test_X.shape[0], 1, test_X.shape[1]))
#y_train = keras.utils.to_categorical(pd.factorize(data.values[:,77])[0],15)
train_y = pd.factorize(train_y)
y_train = keras.utils.to_categorical(train_y[0],7)
y_train = y_train.astype('int')
y_train = np.reshape(y_train,(y_train.shape[0],1,y_train.shape[1]))
test_y = pd.factorize(test_y)
y_test = keras.utils.to_categorical(test_y[0],7)
y_test = y_test.astype('int')
y_test = np.reshape(y_test,(y_test.shape[0],1,y_test.shape[1]))

In [ ]:
import time
data_dim = X_train.shape[2]
timesteps = 1
num_classes = y_train.shape[2]
batch_size = 1

model = Sequential()
model.add(LSTM(50, return_sequences=True, stateful=False, batch_input_shape=(batch_size, timesteps, data_dim)))
model.add(Dropout(0.1))
model.add(LSTM(50, return_sequences=True, stateful=False))
model.add(Dropout(0.1))
model.add(LSTM(50, stateful=True))
model.add(Dropout(0.1))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


start = time.time()

start_time = time.time()
model.fit(X_train, y_train,batch_size=batch_size, epochs=2, shuffle=False)
finish_time = time.time() - start_time
model.save("/content/drive/My Drive/final_model_NSL-KDD_brute_Multiclass.h5")

In [ ]:
import time
#train_object_num=len(train_data)
#print(test[0])
#model training
batch_size = 512
model=Sequential()
model.add(LSTM(32,input_dim=X_train.shape[2],return_sequences=True,kernel_initializer='uniform',activation='relu'))
model.add(LSTM(32,kernel_initializer='uniform',return_sequences=True,activation='relu'))
#model.add(LSTM(128,kernel_initializer='uniform',activation='relu'))
#model.add(Dense(256,activation='relu'))
#model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
#model.add(Dense(16,activation='relu'))
model.add(Dense(y_train.shape[2],activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='NADAM',metrics=['accuracy'])
model.summary()
start_time = time.time()
history = model.fit(X_train,y_train,validation_split=.3,epochs=100,batch_size=batch_size,verbose=1)
finish_time = time.time() - start_time
model.save("/content/drive/My Drive/model_CICIDS_brute_data_valsplit.3.h5")

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, None, 32)          13056     
_________________________________________________________________
lstm_10 (LSTM)               (None, None, 32)          8320      
_________________________________________________________________
dense_9 (Dense)              (None, None, 64)          2112      
_________________________________________________________________
dense_10 (Dense)             (None, None, 7)           455       
Total params: 23,943
Trainable params: 23,943
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
2710/2710 [==============================] - 26s 8ms/step - loss: nan - accuracy: 0.7962 - val_loss: nan - val_accuracy: 0.8034
Epoch 2/100
2710/2710 [==============================] - 23s 8ms/step - loss: nan - accuracy: 0.8033 - val_l

In [ ]:
loss, accuracy = model.evaluate(X_train,y_train)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred = model.predict_classes(X_test)

print("execution time :  ",finish_time)

61923/61923 [==============================] - 95s 2ms/step - loss: nan - accuracy: 0.8030

Loss: nan, Accuracy: 80.30%


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


execution time :   2384.4020709991455


In [ ]:
y_test

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix,  roc_auc_score, roc_curve
import sklearn.metrics as metrics
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
score = model.evaluate(X_test, y_test, batch_size)
y_pred = model.predict(X_test, batch_size= batch_size)
y_pred = y_pred.astype('int')
#y_pred = [np.round(x) for x in y_pred]
#y_pred = np.array(y_pred)
y2 = np.array(test_y[0])

#print(y2)
print(classification_report(y_test,y_pred,"multiclass"))
cm = confusion_matrix(y2.argmax(axis=1), y_pred.argmax(axis=1))
print(cm)


probs = model.predict_proba(X_test,batch_size=batch_size)
#y_preds = np.transpose([pred[:, 1] for pred in probs])
#y_preds=y_preds.reshape(1,-1)
fpr, tpr, threshold = metrics.roc_auc_score(y_score==np.transpose(y_test),y_true= np.transpose(probs),multi_class='ovo')
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/lstm1layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
#csv_logger = CSVLogger('training_set.csv',separator=',', append=False)
start_time = time.time()
model.fit(X_train, y_train, batch_size=batch_size, epochs=20, validation_split=.2)#,callbacks=[checkpointer,csv_logger]
finish_time = time.time() - start_time


Epoch 1/20


ValueError: ignored

In [ ]:
#saving model
model.save("/content/drive/MyDrive/NSL-KDD_results_3LSTM_1Dense_20epoch_1batch_model.h5")

In [ ]:
from keras.models import load_model
#model = load_model('/content/drive/MyDrive/NSL-KDD_results_SimpleRNN_model.hdf5')
loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))

print("--- %s seconds ---",finish_time)
y_pred = model.predict_classes(X_train)
#np.savetxt('/content/drive/MyDrive/NSL-KDD_results_SimpleRNN_predicted.txt', np.transpose([y_test,y_pred]), fmt='%s')

705/705 [==============================] - 1s 996us/step - loss: 1.5715 - accuracy: 0.1944

Loss: 1.57, Accuracy: 19.44%
--- %s seconds --- 10.655580997467041


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
print(model)